# Data Visualization of Food Trucks in IN with Route between.

In [174]:
import plotly.graph_objects as go
import pandas as pd
import json
with open('dev/tokens.json') as f:
    data = json.load(f)
    GOOGLE_API_KEY = data['GOOGLE_API_KEY']
    MAP_TOKEN = data['MAP_TOKEN']


truck_df = pd.read_csv('data/food_trucks.csv')
# mapbox_access_token = open(".mapbox_token").read()
latitudePts = truck_df['lat']
longitudePts = truck_df['lng']
names = truck_df['name']
techpoint = {
    "location": {
        'lat': 39.78728396765869,
        'lng': -86.18316063589883}
}

## Create Tour


In [175]:
df = pd.read_csv('data/directions_driving.csv')

df2 = df.copy()
import random
trucks = df['origin'].unique()
r = random.randint(0, len(trucks)-1)
stops = 10 # how many food trucks you want to visit in a day
start = trucks[r] # you can type the name or use random to select
tour = [start]
for i in range(stops):
    # find the nearest next stop
    df_tmp = df2[df2['origin'] == start].sort_values(by="duration", ascending=True).reset_index(drop=True).iloc[0]
    start = df_tmp["destination"]
    tour.append(start)
    df2 = df2.where(df2['destination'] != start)
tour

['Pico de Gallo Food Truck',
 'El Rinconcito Salvadoreño food truck',
 'Pico de Gallo Food Truck',
 'El Viejon Food Truck',
 'Food Truck Don Luis',
 'KG Slider Station',
 'TACOPOLIS (LA CAPITAL DEL TACO)',
 'Antojitos Guzman food truck #1',
 'Tacos El Iturbidense',
 'El Amigo Taqueria Food Truck',
 'FELIX TACOS TRUCK']

## Get route data

In [176]:
with open('data/full_data.json') as f:
    details = json.load(f)


def get_route_between(origin, destination):
    for i in details:
        if i["origin"] ==origin and i["destination"] ==destination:
            return i

lats = []
lngs = []
dists = []
times = []
for i in range(len(tour)):
    if i == len(tour)-1:
        break
    origin = tour[i]
    destination = tour[i+1]
    result_dict = get_route_between(origin, destination)
    lats = lats + result_dict["lats"]
    lngs = lngs + result_dict["lngs"]
    dists.append(result_dict['distance'])
    times.append(result_dict['duration'])

In [177]:
## Visualize with Interactive Dashboard

In [178]:
fig = go.Figure(go.Scattermapbox(
    lat=lats,
    lon=lngs,
    mode='lines',
    line=dict(width=2),
    marker=dict(
        size=5,
    ),
    text=[round(t/60,1) for t in times],
    hoverinfo='text',
    name = "Route",
))
fig.add_trace(go.Scattermapbox(
    lat=latitudePts,
    lon=longitudePts,
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=9
    ),
    text=names,
    hoverinfo='text',
    name = 'Trucks',
    
))
fig.update_traces(textposition='top center')
fig.update_traces(showlegend=True)
fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=MAP_TOKEN,
        bearing=0,
        center=dict(
            lat=techpoint['location']['lat'],
            lon=techpoint['location']['lng']
        ),
        pitch=0,
        zoom=10
    ),
    title_text = f"Total Distance: {round(sum(dists)/1000,1)} Km\n Total Distance: {round(sum(times)/60,1)} Min"
)


fig.show()